In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.maf.db as db
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import copy

/Users/yoachim/lsst/DarwinX86/anaconda/2.2.0/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
runNames = ['enigma_1189', 'minion_1016']
outDir = 'astrometry_test'
resultsDb = db.ResultsDb(outDir=outDir)
filters = ['u', 'g', 'r', 'i', 'z', 'y']

In [3]:
for runName in ['enigma_1189']:
    opsdb = db.OpsimDatabase(runName+'_sqlite.db')
    propids, propTags = opsdb.fetchPropInfo()
    WFDpropid = propTags['WFD'][0]
    slicer = slicers.UniSlicer()
    bundleList = []
    for filterName in filters:
        sql = 'propID = %i and filter="%s"' % (WFDpropid, filterName)
        bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='filtSkyBrightness'), slicer, sql))
        bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='fiveSigmaDepth'), slicer, sql))
        bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='airmass'), slicer, sql))
        if runName == 'enigma_1189':
            bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='finSeeing'), slicer, sql))
        else:
            bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='FWHMeff'), slicer, sql))
    bundleDict = metricBundles.makeBundlesDictFromList(bundleList)
    bgroup = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
    bgroup.runAll()
    print '--------'
    print runName
    for bundle in bundleList:
        print bundle.constraint, bundle.metric.name, bundle.metricValues
    print '--------'
enigma_BL = copy.deepcopy(bundleList)

Querying database with constraint propID = 364 and filter="z"
Found 417684 visits
Running:  ['opsim_Median_airmass_propID_364_and_z_UNIS', 'opsim_Median_finSeeing_propID_364_and_z_UNIS', 'opsim_Median_fiveSigmaDepth_propID_364_and_z_UNIS', 'opsim_Median_filtSkyBrightness_propID_364_and_z_UNIS']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database with constraint propID = 364 and filter="g"
Found 203973 visits
Running:  ['opsim_Median_filtSkyBrightness_propID_364_and_g_UNIS', 'opsim_Median_fiveSigmaDepth_propID_364_and_g_UNIS', 'opsim_Median_finSeeing_propID_364_and_g_UNIS', 'opsim_Median_airmass_propID_364_and_g_UNIS']
Completed metric generation.
Running reduce methods.
Running summary statistics.
Completed.
Querying database with constraint propID = 364 and filter="i"
Found 464616 visits
Running:  ['opsim_Median_filtSkyBrightness_propID_364_and_i_UNIS', 'opsim_Median_airmass_propID_364_and_i_UNIS', 'opsim_Median_fiveSigmaDepth_

AttributeError: 'list' object has no attribute 'copy'

In [ ]:
for runName in ['minion_1016']:
    opsdb = db.OpsimDatabase(runName+'_sqlite.db')
    propids, propTags = opsdb.fetchPropInfo()
    WFDpropid = propTags['WFD'][0]
    slicer = slicers.UniSlicer()
    bundleList = []
    for filterName in filters:
        sql = 'propID = %i and filter="%s"' % (WFDpropid, filterName)
        bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='filtSkyBrightness'), slicer, sql))
        bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='fiveSigmaDepth'), slicer, sql))
        bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='airmass'), slicer, sql))
        if runName == 'enigma_1189':
            bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='finSeeing'), slicer, sql))
        else:
            bundleList.append(metricBundles.MetricBundle(metrics.MedianMetric(col='FWHMeff'), slicer, sql))
    bundleDict = metricBundles.makeBundlesDictFromList(bundleList)
    bgroup = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)
    bgroup.runAll()
    print '--------'
    print runName
    for bundle in bundleList:
        print bundle.constraint, bundle.metric.name, bundle.metricValues
    print '--------'